In [1]:
import numpy as np

import gsd.hoomd
import coxeter
import freud

#### creating unit volume particles of truncated tetrahedron with initial truncation at 0.5

In [2]:
alpha_init = 0.5
platonic = coxeter.families.TruncatedTetrahedronFamily()
particle = platonic.get_shape(truncation=alpha_init)
verts = particle.vertices/particle.volume**(1/3)

new_particle = coxeter.shapes.ConvexPolyhedron(verts)
new_particle.volume

1.0000000000000002

#### target structure of FCC

In [3]:
uc = freud.data.UnitCell.fcc()

box, position = uc.generate_system(num_replicas=2, scale=2.3)
position+=0.5

N_particles = len(position)

#### creating snapshot

In [4]:
snapshot = gsd.hoomd.Frame()
snapshot.particles.types = ['A']
snapshot.particles.N = N_particles
snapshot.particles.typeid = [0] * N_particles
snapshot.particles.position = position[:N_particles]
snapshot.particles.orientation = [1, 0, 0, 0]*N_particles

verts = verts.tolist()
snapshot.particles.type_shapes = [{'type': 'ConvexPolyhedron',
                                   'vertices': verts}]

snapshot.configuration.box = [box.Lx, box.Ly, box.Lz, box.xy, box.xz, box.yz]

#### save snapshot

In [5]:
with gsd.hoomd.open(name='./DATA/lattice.gsd', mode='wb') as f:
    f.append(snapshot)